In [1]:
import numpy as np
import torch

In [2]:
#parameter setting
num_hidden_state = 3
num_obs = 5
length = 3
num_samples = 1

In [3]:
#define some useful functions
def generate_HMM_params(num_hidden_state, num_obs):
    # random generate the transition matrix and observation matrix, and compute the stationary distribution
    
    alpha_state = np.ones(num_hidden_state)
    alpha_obs = np.ones(num_obs) / num_obs
    trans_mat = np.random.dirichlet(alpha_state, num_hidden_state)
    obs_mat = np.random.dirichlet(alpha_obs, num_hidden_state)
    tmp = np.ones((num_hidden_state + 1, num_hidden_state))
    tmp[:-1] = np.identity(num_hidden_state) - trans_mat.T
    tmp_v = np.zeros(num_hidden_state + 1)
    tmp_v[-1] = 1
    stat_dist = np.linalg.lstsq(tmp, tmp_v, rcond=None)[0]
    return trans_mat, obs_mat, stat_dist

In [4]:
trans_mat, obs_mat, stat_dist = generate_HMM_params(num_hidden_state, num_obs) # generate parameters for HMM
print(trans_mat)
print(obs_mat)
print(stat_dist)
print(stat_dist @ trans_mat)

[[0.31171752 0.14466132 0.54362116]
 [0.48262276 0.41041321 0.10696403]
 [0.63493906 0.18155131 0.18350963]]
[[5.31183612e-03 3.18727100e-01 4.60386268e-01 1.87884837e-01
  2.76899589e-02]
 [1.39397202e-03 1.01159214e-04 8.84424090e-01 2.97764010e-02
  8.43043777e-02]
 [3.88117700e-03 2.36483698e-01 1.76779583e-02 7.20955470e-01
  2.10016963e-02]]
[0.45524953 0.21365455 0.33109592]
[0.45524953 0.21365455 0.33109592]


In [5]:
def generate_HMM_sequences(trans_mat, obs_mat, init_dist, length, num_samples = 1):
    # generate sample sequences from HMM
    
    states = np.zeros((num_samples, length))
    obs = np.zeros((num_samples, length))
    tmp_state = np.argmax(np.random.multinomial(1, init_dist, num_samples), axis = 1)
    #print(tmp_state)
    for i in range(length):
        #print("i: ", i)
        states[:, i] = tmp_state
        for j in range(num_samples):
            obs[j, i] = np.random.multinomial(1, obs_mat[tmp_state[j]]).argmax()
            tmp_state[j] = np.random.multinomial(1, trans_mat[tmp_state[j]]).argmax()
        #print("obs[:, i]: ", obs[:, i])
    return states, obs

In [10]:
test_states, test_obs = generate_HMM_sequences(trans_mat, obs_mat, stat_dist, length, 100000)
lst = [0, 0, 0, 0, 0]
for i in range(test_states.shape[0]):
    if (test_states[i, 0] == 0.0): # and (test_states[i, 1] == 2.0):
        lst[int(test_obs[i, 0])] += 1
print(lst)
lst = np.array(lst).astype('float64')
lst /= lst.sum()
print(lst)

[213, 14394, 21011, 8542, 1279]
[0.0046876  0.31677634 0.46240014 0.18798829 0.02814763]


In [11]:
def forward_compute(trans_mat, obs_mat, init_dist, obs_to_pos):
    # compute \sum_{h_1,...,h_{pos-1}} P(h_1,...,h_{pos},x_1,...,x_{pos-1})
    pos = obs_to_pos.shape[0] + 1
    num_hidden_state = trans_mat.shape[0]
    num_obs = obs_mat.shape[1]
    forward = np.zeros((pos, num_hidden_state))
    forward[0] = init_dist
    for i in range(1, pos):
        for j in range(num_hidden_state):
            for k in range(num_hidden_state):
                #print(i, j, k)
                #print(forward[i - 1, k], trans_mat[k, j], obs_mat[k, int(obs_to_pos[i - 1])])
                forward[i, j] += forward[i - 1, k] * trans_mat[k, j] * obs_mat[k, int(obs_to_pos[i - 1])]
    #print("forward: ", forward)
    return forward[pos - 1]

In [12]:
def backward_compute(trans_mat, obs_mat, obs_from_pos):
    num_hidden_state = trans_mat.shape[0]
    num_obs = obs_mat.shape[1]
    back_length = obs_from_pos.shape[0]
    if (back_length == 0):
        return np.ones(num_hidden_state)
    backward = np.zeros((back_length, num_hidden_state))
    for j in range(num_hidden_state):
         for k in range(num_hidden_state):
            backward[0, j] += trans_mat[j, k] * obs_mat[k, int(obs_from_pos[-1])]
    for i in range(1, back_length):
        for j in range(num_hidden_state):
            for k in range(num_hidden_state):
                backward[i, j] += trans_mat[j, k] * obs_mat[k, int(obs_from_pos[-(i + 1)])] * backward[i - 1, k]
    #print("backward: ", backward)
    return backward[-1]

In [13]:
def x_i_conditional_prob(trans_mat, obs_mat, init_dist, known_X, pos):
    num_hidden_state = trans_mat.shape[0]
    num_obs = obs_mat.shape[1]
    num_samples = known_X.shape[0]
    length = known_X.shape[1]
    x_pos_conditional_prob = np.zeros((num_samples, num_obs))
    h_pos_conditional_prob = np.zeros((num_samples, num_hidden_state))
    for i in range(num_samples):
        #print("x_i_conditional_prob: i=", i)
        sample_obs_vec = known_X[i]
        forward_vec = forward_compute(trans_mat, obs_mat, init_dist, known_X[i, :pos[i]])
        backward_vec = backward_compute(trans_mat, obs_mat, known_X[i, pos[i] + 1:])
        #print("forward_vec: ", forward_vec)
        #print("backward_vec: ", backward_vec)
        h_prob_tmp = forward_vec * backward_vec
        tmp = h_prob_tmp.sum()
        h_prob_tmp /= tmp
        h_pos_conditional_prob[i] = h_prob_tmp
        x_pos_conditional_prob[i] = h_prob_tmp @ obs_mat
    return h_pos_conditional_prob, x_pos_conditional_prob

In [15]:
test_states, test_obs = generate_HMM_sequences(trans_mat, obs_mat, stat_dist, length, 100000)
lst = [0, 0, 0]
for i in range(test_states.shape[0]):
    if (test_states[i, 0] == 0.0) and (test_states[i, 1] == 2.0):
        lst[int(test_states[i, 2])] += 1
print(lst)
lst = np.array(lst).astype('float64')
lst /= lst.sum()
print(lst)

[15760, 4543, 4542]
[0.63433286 0.18285369 0.18281344]


In [17]:
states, obs = generate_HMM_sequences(trans_mat, obs_mat, stat_dist, length, num_samples)
states, obs

(array([[0., 0., 2.]]), array([[1., 2., 3.]]))

In [18]:
trans_mat, obs_mat, stat_dist = generate_HMM_params(num_hidden_state, num_obs) # generate parameters for HMM

#obs_mat = np.zeros((num_hidden_state, num_obs))
#obs_mat[:, :num_hidden_state] = np.identity(num_hidden_state)

states, obs = generate_HMM_sequences(trans_mat, obs_mat, stat_dist, length, num_samples) # generate sample sequences

pos = np.random.randint(length, size = num_samples)
#pos_one_hot = np.zeros((num_samples, length))
#tmp = np.arange(num_samples)
#pos_one_hot[tmp, pos] = 1
#print(pos_one_hot)
print("transition matrix")
print(trans_mat)
print("observation matrix")
print(obs_mat)
print("stationary distribution")
print(stat_dist)
print("states and observations, first half of each row is states")
print(np.concatenate((states, obs), axis = 1))
print("positions: ", pos)
h, x = x_i_conditional_prob(trans_mat, obs_mat, stat_dist, obs, pos)
print("Pr[H_i|x_-i], j-th row is for j-th sample and i=positions[j]:")
print(h)
print("Pr[X_i|x_-i], j-th row is for j-th sample and i=positions[j]:")
print(x)

transition matrix
[[0.01554627 0.14416745 0.84028628]
 [0.13160845 0.43043007 0.43796147]
 [0.3378113  0.01195339 0.65023531]]
observation matrix
[[5.10336443e-05 4.22640901e-06 2.08226911e-14 1.80774938e-01
  8.19169802e-01]
 [8.86951621e-01 1.03226795e-01 2.03990342e-03 1.96728311e-03
  5.81439722e-03]
 [3.78728552e-01 1.24232902e-01 1.81257702e-03 4.50485680e-01
  4.47402888e-02]]
stationary distribution
[0.24363546 0.07594781 0.68041673]
states and observations, first half of each row is states
[[2. 2. 2. 0. 0. 0.]]
positions:  [2]
Pr[H_i|x_-i], j-th row is for j-th sample and i=positions[j]:
[[0.28089022 0.12744663 0.59166315]]
Pr[X_i|x_-i], j-th row is for j-th sample and i=positions[j]:
[[0.33713306 0.08666112 0.00133241 0.31756441 0.25730899]]


In [20]:
#x_i_conditional_prob(trans_mat, obs_mat, stat_dist, obs, np.array([1]))
test_states, test_obs = generate_HMM_sequences(trans_mat, obs_mat, stat_dist, length, 1000000)
lst = [0, 0, 0, 0, 0]
for i in range(test_states.shape[0]):
    if (test_obs[i, 0] == 0.0) and (test_obs[i, 1] == 0.0):
        lst[int(test_states[i, 2])] += 1
print(lst)
lst = np.array(lst).astype('float64')
lst /= lst.sum()
print(lst)

[28774, 12854, 61110, 0, 0]
[0.28007164 0.12511437 0.59481399 0.         0.        ]


In [10]:
h @ obs_mat, x

(array([[0.26064652, 0.33029826, 0.11729067, 0.04206999, 0.24969456]]),
 array([[0.26064652, 0.33029826, 0.11729067, 0.04206999, 0.24969456]]))

In [21]:
from torch.utils import data

In [22]:
#parameter setting
num_hidden_state = 3
num_obs = 5
length = 10
num_samples = 1000

In [23]:
trans_mat, obs_mat, stat_dist = generate_HMM_params(num_hidden_state, num_obs) # generate parameters for HMM

states, obs = generate_HMM_sequences(trans_mat, obs_mat, stat_dist, length, num_samples) # generate sample sequences

pos = np.random.randint(length, size = num_samples)

print("transition matrix")
print(trans_mat)
print("observation matrix")
print(obs_mat)
print("stationary distribution")
print(stat_dist)
print("states and observations, first half of each row is states")
print(np.concatenate((states, obs), axis = 1))
print("positions: ", pos)
h, x = x_i_conditional_prob(trans_mat, obs_mat, stat_dist, obs, pos)
print("Pr[H_i|x_-i], j-th row is for j-th sample and i=positions[j]:")
print(h)
print("Pr[X_i|x_-i], j-th row is for j-th sample and i=positions[j]:")
print(x)

transition matrix
[[0.42265632 0.22411211 0.35323157]
 [0.46122757 0.40080087 0.13797155]
 [0.04676753 0.5609235  0.39230897]]
observation matrix
[[2.92681512e-01 1.50206764e-04 6.16017949e-01 1.42344315e-03
  8.97268893e-02]
 [1.46534926e-02 2.00898243e-04 1.85346850e-01 7.24383169e-05
  7.99726321e-01]
 [2.77131417e-06 1.92761368e-01 1.11532894e-05 3.91880687e-01
  4.15344020e-01]]
stationary distribution
[0.33200907 0.38711344 0.28087749]
states and observations, first half of each row is states
[[0. 2. 1. ... 0. 0. 3.]
 [2. 1. 2. ... 4. 4. 4.]
 [1. 0. 0. ... 3. 4. 4.]
 ...
 [0. 2. 1. ... 4. 2. 2.]
 [0. 2. 1. ... 2. 2. 2.]
 [2. 1. 0. ... 1. 1. 4.]]
positions:  [2 0 9 2 9 2 3 6 5 1 5 2 4 7 7 6 8 6 3 0 5 6 7 2 5 1 8 5 2 6 4 5 3 5 7 3 9
 1 3 9 1 0 2 2 6 9 4 3 9 1 5 8 8 1 8 1 4 3 6 8 3 0 4 6 7 2 9 9 6 4 1 7 7 9
 7 8 2 6 3 5 9 8 1 5 3 1 5 2 9 7 8 6 8 1 6 6 3 0 2 8 0 2 2 9 1 3 5 1 0 6 9
 6 3 1 3 7 4 5 2 2 9 5 9 2 8 9 8 5 1 6 4 0 3 4 0 0 7 8 5 3 0 8 8 8 9 1 5 2
 9 3 7 9 2 5 4 1 6 5 3 1 1 8

In [26]:
features, labels = x, h

In [169]:
# Model parameters.
lr = 3
epochs = 1000
batch_size = 100

In [170]:
dataset = data.TensorDataset(torch.FloatTensor(x), torch.FloatTensor(h))
train_dl = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=False)

In [171]:
from torch import nn

In [172]:
net = nn.Sequential(nn.Linear(num_obs, num_hidden_state, bias=False))

In [173]:
loss = nn.MSELoss()

In [174]:
trainer = torch.optim.SGD(net.parameters(), lr=lr)

In [176]:
for i in range(epochs):
    total_loss = 0
    for X, y in train_dl:
        l = loss(net(X) ,y)
        total_loss += l
        trainer.zero_grad()
        l.backward()
        trainer.step()
    if (i % 100 == 0):
        print("epoch: ", i)
        print("loss: ", total_loss)

epoch:  0
loss:  tensor(0.1683, grad_fn=<AddBackward0>)
epoch:  100
loss:  tensor(3.5754e-09, grad_fn=<AddBackward0>)
epoch:  200
loss:  tensor(7.0949e-13, grad_fn=<AddBackward0>)
epoch:  300
loss:  tensor(6.4643e-13, grad_fn=<AddBackward0>)
epoch:  400
loss:  tensor(6.0223e-13, grad_fn=<AddBackward0>)
epoch:  500
loss:  tensor(5.6167e-13, grad_fn=<AddBackward0>)
epoch:  600
loss:  tensor(5.2111e-13, grad_fn=<AddBackward0>)
epoch:  700
loss:  tensor(4.8881e-13, grad_fn=<AddBackward0>)
epoch:  800
loss:  tensor(4.5680e-13, grad_fn=<AddBackward0>)
epoch:  900
loss:  tensor(4.2430e-13, grad_fn=<AddBackward0>)


In [177]:
net[0].weight.data

tensor([[ 0.6292, -0.2160,  1.3714,  0.4553, -0.3293],
        [-0.3151, -0.3703, -0.0316, -1.1572,  1.2637],
        [-0.1060,  0.9090,  0.0466,  2.1145, -0.0093]])

In [178]:
np.linalg.pinv(obs_mat)

array([[ 0.70946001, -0.25856422,  0.16254959],
       [ 0.13386227, -0.53187938,  1.00319817],
       [ 1.33246609, -0.05937896, -0.08454823],
       [ 0.27525784, -1.08351973,  2.04118875],
       [-0.32187423,  1.26915907,  0.01617982]])

In [181]:
for X, y in train_dl:
    print(X[:9])
    print(y[:9])
    print((X @ net[0].weight.data.T - y)[:9])
    print((X.double() @ torch.from_numpy(np.linalg.pinv(obs_mat)))[:9])
    print(net[0].weight.data.T.double() - torch.from_numpy(np.linalg.pinv(obs_mat)))
    break

tensor([[0.1066, 0.0231, 0.3084, 0.0471, 0.5148],
        [0.1127, 0.0271, 0.3145, 0.0553, 0.4905],
        [0.1092, 0.0400, 0.2982, 0.0816, 0.4709],
        [0.0881, 0.0642, 0.2451, 0.1308, 0.4717],
        [0.1092, 0.0400, 0.2982, 0.0816, 0.4709],
        [0.1692, 0.0145, 0.4127, 0.0300, 0.3736],
        [0.1404, 0.0817, 0.3112, 0.1665, 0.3002],
        [0.1062, 0.0871, 0.2537, 0.1774, 0.3757],
        [0.1485, 0.0562, 0.3453, 0.1147, 0.3353]])
tensor([[0.3370, 0.5441, 0.1189],
        [0.3599, 0.5005, 0.1397],
        [0.3511, 0.4420, 0.2069],
        [0.2819, 0.3855, 0.3327],
        [0.3511, 0.4420, 0.2069],
        [0.5599, 0.3657, 0.0744],
        [0.4744, 0.1024, 0.4232],
        [0.3529, 0.1957, 0.4513],
        [0.4966, 0.2125, 0.2909]])
tensor([[ 1.1921e-07, -2.3842e-07,  2.7567e-07],
        [ 8.9407e-08, -2.3842e-07,  2.0862e-07],
        [ 5.9605e-08, -1.1921e-07,  1.0431e-07],
        [ 0.0000e+00,  5.9605e-08, -5.9605e-08],
        [ 0.0000e+00, -1.1921e-07,  8.9407e-08

In [109]:
x, h

(array([[0.10659928, 0.02308308, 0.30843146, 0.04712147, 0.51476471],
        [0.11266594, 0.02707417, 0.3144569 , 0.05527557, 0.49052742],
        [0.10924114, 0.04002508, 0.2982135 , 0.08161441, 0.47090587],
        ...,
        [0.12111725, 0.01954281, 0.33390811, 0.0399925 , 0.48543933],
        [0.08804524, 0.07371188, 0.23693078, 0.15006043, 0.45125167],
        [0.08846542, 0.0729027 , 0.23826504, 0.14841584, 0.45195101]]),
 array([[0.3369734 , 0.54410674, 0.11891986],
        [0.35988624, 0.50046146, 0.13965229],
        [0.35111199, 0.44198166, 0.20690635],
        ...,
        [0.38822297, 0.51122894, 0.10054809],
        [0.28409307, 0.33407683, 0.38183009],
        [0.28538292, 0.33698888, 0.3776282 ]]))

In [116]:
np.linalg.pinv(obs_mat)

array([[ 0.70946001, -0.25856422,  0.16254959],
       [ 0.13386227, -0.53187938,  1.00319817],
       [ 1.33246609, -0.05937896, -0.08454823],
       [ 0.27525784, -1.08351973,  2.04118875],
       [-0.32187423,  1.26915907,  0.01617982]])

In [182]:
states, obs = generate_HMM_sequences(trans_mat, obs_mat, stat_dist, length, num_samples) # generate sample sequences

pos = np.random.randint(length, size = num_samples)

print("transition matrix")
print(trans_mat)
print("observation matrix")
print(obs_mat)
print("stationary distribution")
print(stat_dist)
print("states and observations, first half of each row is states")
print(np.concatenate((states, obs), axis = 1))
print("positions: ", pos)
h, x = x_i_conditional_prob(trans_mat, obs_mat, stat_dist, obs, pos)

transition matrix
[[0.42265632 0.22411211 0.35323157]
 [0.46122757 0.40080087 0.13797155]
 [0.04676753 0.5609235  0.39230897]]
observation matrix
[[2.92681512e-01 1.50206764e-04 6.16017949e-01 1.42344315e-03
  8.97268893e-02]
 [1.46534926e-02 2.00898243e-04 1.85346850e-01 7.24383169e-05
  7.99726321e-01]
 [2.77131417e-06 1.92761368e-01 1.11532894e-05 3.91880687e-01
  4.15344020e-01]]
stationary distribution
[0.33200907 0.38711344 0.28087749]
states and observations, first half of each row is states
[[0. 0. 0. ... 4. 4. 4.]
 [2. 2. 2. ... 4. 3. 4.]
 [2. 1. 1. ... 4. 1. 4.]
 ...
 [0. 0. 1. ... 2. 4. 4.]
 [1. 1. 2. ... 4. 4. 4.]
 [2. 1. 0. ... 4. 4. 4.]]
positions:  [9 7 6 7 1 1 5 5 4 3 4 0 8 0 0 7 1 3 3 1 7 8 3 7 1 0 9 8 0 6 2 0 0 9 4 4 8
 0 8 1 9 4 3 3 3 4 9 2 6 4 1 7 0 0 6 8 5 4 5 1 5 9 1 7 7 7 9 7 4 8 9 8 2 3
 6 6 7 8 3 6 6 3 2 7 0 9 8 6 8 4 6 1 7 4 2 8 9 1 9 8 8 1 6 5 6 1 3 6 6 7 9
 3 5 0 6 4 6 3 0 6 3 8 9 0 0 4 6 7 8 9 5 4 6 3 2 8 6 0 9 2 4 0 1 5 3 8 6 2
 6 6 3 4 9 7 7 5 1 4 4 0 2 3

In [186]:
torch.from_numpy(x) @ net[0].weight.data.T.double() - torch.from_numpy(h)

tensor([[ 5.2449e-09, -8.7515e-08,  9.4320e-08],
        [-2.3336e-07,  4.2445e-07, -3.8074e-07],
        [-1.5459e-07,  2.8853e-07, -2.7494e-07],
        ...,
        [-7.1988e-08,  8.2092e-09,  4.8466e-08],
        [-2.2446e-08,  1.4187e-08, -2.6014e-08],
        [-1.3576e-07,  2.3927e-07, -2.2379e-07]], dtype=torch.float64)